In [22]:
import xarray as xr
from pyclim_noresm.aerosol_feedbacks import (partialDF_partialDC,
                                            partialDC_partialDT)
import yaml

params = snakemake.params
time_slice = params.get('time_slice',slice(5,None))
piControl_slice = params.get('piControl_slice', slice(120,150))

In [3]:
emis_2xCO2 = xr.open_dataset(snakemake.input.emis_2xCO2[0])

In [4]:
emis_exp = xr.open_dataset(snakemake.input.emis_exp[0]).isel(time=time_slice)
emis_ctrl = xr.open_dataset(snakemake.input.emis_ctrl[0]).isel(time=time_slice)
emis_piCtrl = xr.open_dataset(snakemake.input.emis_PIctrl[0]).isel(time=piControl_slice)
emis_2xCO2 = xr.open_dataset(snakemake.input.emis_2xCO2[0]).isel(time=piControl_slice)
t_piCtrl = xr.open_dataset(snakemake.input.t_ctrl[0]).isel(time=piControl_slice)
t_exp = xr.open_dataset(snakemake.input.t_exp[0]).isel(time=piControl_slice)
area_cello = xr.open_dataset(snakemake.input.area_cello)
forcing = xr.open_dataset(snakemake.input.forcing).isel(year=time_slice)

In [5]:
with xr.set_options(keep_attrs=True):
    forcing = forcing
    deltaC_piClim = emis_exp[emis_exp.variable_id]-emis_ctrl[emis_ctrl.variable_id]
    deltaC_piControl = emis_2xCO2[emis_2xCO2.variable_id]-emis_piCtrl[emis_piCtrl.variable_id].assign_coords(time=emis_2xCO2.time)
    deltaT = t_exp[t_exp.variable_id] - t_piCtrl[t_piCtrl.variable_id].assign_coords(time=t_exp.time)

In [6]:
dFdC, deltaCPiclim, erf= partialDF_partialDC(forcing, deltaC_piClim, area_cello, per_year=False)

In [7]:
dCdT, deltaC_piControl, deltat = partialDC_partialDT(deltaT, deltaC_piControl,area_cello,per_year=False)

In [35]:
dCdT

In [34]:
dFdC

In [31]:
data = {}
data['dCdT']={}
data['dCdT']['units'] = 'Tg year-1/K'
data['dCdT']['data'] = (dCdT.values*1e-9).tolist()
data['dFdC']= {}
data['dFdC']['units'] = 'W m-2/tg year-1'
data['dFdC']['data'] = (dFdC.values*1e9).tolist()
data['delta_ems'] = {}
data['delta_ems']['data'] = (deltaCPiclim.values*1e-9).tolist()
data['delta_ems']['units'] = 'Tg year-1'
data['dT_dEms'] = {}
data['dT_dEms']['units'] = 'Tg year-1'
data['dT_dEms']['data'] = (deltaC_piControl.values*1e-9).tolist()
data['alpha']['units'] = 'W m-2 K-1'
data['alpha']['data']

In [33]:
data

In [32]:
with open(snakemake.output.outpath, 'w') as f:
    yaml.safe_dump(data,f,default_flow_style=False)

In [29]:
dFdC